##First load the data
To begin, load up some data from a database into a DataFrame.

In [ ]:
#To keep the page organized, we will do all the imports here
import pandas as pd
from scipy import stats
import re
from datetime import datetime
import numpy as np


In [ ]:
#Load csv file into a dataframe
fifa_df = pd.read_csv('/content/fifa21 raw data v2.csv')

<ipython-input-277-41ea178855d2>:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_df = pd.read_csv('/content/fifa21 raw data v2.csv')


First, we will quickly examine our dataframe

In [ ]:
fifa_df.shape

(18979, 77)

In [ ]:
fifa_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

In [ ]:
fifa_df.head(5)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


Before we begin our analysis, lets first state the guiding questions we will be looking at.


*   Do the height and weight columns have the appropriate data type?
*   Can we separate the joined column into year, month and day columns?
*   Can we clean and transform the value, wage and release clause columns into columns of integers?
*   How can we remove the newline characters from the Hits column?
*   Should we seoarate the Team & Contract column into separate team and contract columns?








##Question 1: *Do the height and weight columns have the appropriate data types?*

We want both height and weight to be integers.

In [ ]:
#Height is written in both centimeter and feet format.
#We will be converting these to integers values that represents total centimeters.

# Function to convert height to centimeters
def convert_height(height):
    cm_pattern = r"(\d+)cm"
    feet_pattern = r"(\d+)'(\d+)\""

    cm_match = re.match(cm_pattern, height)
    feet_match = re.match(feet_pattern, height)

    if cm_match:
        return int(cm_match.group(1))
    elif feet_match:
        feet = int(feet_match.group(1))
        inches = int(feet_match.group(2))
        total_cm = feet * 30.48 + inches * 2.54
        return int(total_cm)
    else:
        raise ValueError("Invalid height format: " + height)

# Apply the function to 'Height' column and store the result in 'height_cm' column
fifa_df['height_cm'] = fifa_df['Height'].apply(convert_height)

# Print the updated DataFrame
fifa_df['height_cm']

0        170
1        187
2        188
3        181
4        175
        ... 
18974    178
18975    175
18976    179
18977    175
18978    188
Name: height_cm, Length: 18979, dtype: int64

In [ ]:
#Confirm our values formatted in feet'inches are converted to total cm
fifa_df.iloc[862:872, 77:79]

,height_cm
862,185
863,182
864,185
865,180
866,187
867,182
868,190
869,182
870,184
871,177


From the results above, we see that values in both the height and weight column are integers. Hence, they have the appropriate data types.

##Notes for solution to Question 1

The 're' module in Python provides support for working with regular expressions, which are sequences of characters that define a search pattern. Regular expressions can be used for pattern matching, seaching, and minipulation strings. The 're' module offers a variety of functions and methods for working with regular expressions.

Common 're' Functions/Methods:
1. 're.match(pattern, string)':Checks if the regular expression pattern matches at the beginning of the string.
2. 're.search(pattern, string)': Searches the entire string for a match with the pattern.
3. 're.findall(pattern, string)': Finds all occurences of the pattern in the string and returns them as a list.
4. 're.finditer(pattern, string)': Finds all occurences of the pattern in the string and returns an interator of match objects.
5. 're.sub(pattern, replacement, string)': Replaces occurences of the pattern in the string with the specified replacement.

The '\d+' regular expression matches one or more digits (0-9). Here's what each part means:



*   '\d+': This is a special character in regular expressions representing any digit (0-9).
*   This is a quantifier that means "one or more occurrences of the preceding pattern." In this case, it applies to '\d', indicated one or more digits.




###Question 2: *Can we separate the joined column into year, month and day columns?*

In [ ]:
# Function to parse 'Joined' column and extract month, day, and year
def parse_date(date_str):
    date_object = datetime.strptime(date_str, '%b %d, %Y')
    return date_object.month, date_object.day, date_object.year

# Apply the parse_date function to 'Joined' column and split into separate columns
fifa_df[['Month', 'Day', 'Year']] = fifa_df['Joined'].apply(parse_date).apply(pd.Series)

# Print the updated DataFrame
print(fifa_df[['Month', 'Day', 'Year']])

       Month  Day  Year
0          7    1  2004
1          7   10  2018
2          7   16  2014
3          8   30  2015
4          8    3  2017
...      ...  ...   ...
18974      7   13  2018
18975      8    1  2020
18976      3    8  2019
18977      9   22  2020
18978      7   29  2019

[18979 rows x 3 columns]


###Notes for solution to Question 2


1. The 'parse_date' function uses 'strptime' to parse the date strings in the formal "Jul 1, 2004" where '%b' represents the abbreviated month name, '%d' represents the day and '%Y' represents the 4-digit year.
2. 'apple(parse_date).apply(pd.Series)' applies the 'parse_date' function to the 'Joined' column, created a DataFrame with three columns (month, day, and year) which are then assigned to the respective columns in the original DataFrame.

After running this code, the DataFrame will have separate 'Month', 'Day', and 'Year' columns with the respective values extracted from the original 'Joined' column.

##Question 3: *Can we clean and transform the value, wage and release clause columns into columns of integers?*

We want the values in these columns to be floating numbers. The reason for this instead of having integers is that some are written like 45.1M (as in 45.1 million). If are to have them as integers, it would 45,100,100. To improve readability, we will write this as floating numbers, and the units (millions as m and thousands as m) will be specified in the column names.

###First, lets transform our 'Value' column.

In [ ]:
# Function to convert 'Value' column to numeric format
def convert_value(value_str):
    numeric_part = re.sub(r'[^\d.]', '', value_str)  # Remove non-numeric characters
    if numeric_part:
        return float(numeric_part)  # Convert the numeric part to float
    else:
        raise ValueError("Invalid 'Value' format: " + value_str)

# Apply the function to 'Value' column and store the result in 'value_mm' column
fifa_df['value_mm'] = fifa_df['Value'].apply(convert_value)
#Note if you run this cell again after run all, it will return an error.
#That is because 'Value' no longer exists after the intial run through.

# Drop the original 'Value' column
fifa_df.drop(columns=['Value'], inplace=True)

# Print the updated DataFrame
print(fifa_df['value_mm'])

0        103.5
1         63.0
2        120.0
3        129.0
4        132.0
         ...  
18974    100.0
18975    130.0
18976    120.0
18977    100.0
18978    100.0
Name: value_mm, Length: 18979, dtype: float64


###Notes for above cell


In the code above, the 're.sub(r'[^\d.]', '', value_str) lines removes all characters from the 'Value' column except digits and dots. This way, it attempts to extract any numeric value, regardless of the special characters used. The result is then converted to a float.

1. re.sub(r'[^\d.]', '', value_str)
re.sub() is a function from the re module that performs a substitution operation on a string using regular expressions.
r'[^\d.]' is a regular expression pattern. Here, r denotes a raw string, which helps in interpreting backslashes (\) as literal characters.
[^\d.] is a character class that matches any character except digits (\d) and dots (.). The ^ inside the square brackets negates the character class, meaning it matches any character NOT in the specified set.
'' is the replacement string, meaning any character that matches the pattern will be replaced with an empty string, effectively removing it.
This line removes all non-numeric characters from the value_str. For example, if value_str is '€103.5M', re.sub(r'[^\d.]', '', value_str) will result in '103.5'.

###Second, lets transform the 'Wage' column.

In [ ]:
# Function to convert 'Wage' column to numeric format
def convert_wage(wage_str):
    # Remove non-numeric characters and 'K', then convert to float
    numeric_part = re.sub(r'[^\d.]|K', '', wage_str)
    if numeric_part:
        return float(numeric_part)  # Keep the wage amount in thousands (no multiplication)
    else:
        raise ValueError("Invalid 'Wage' format: " + wage_str)

# Apply the function to 'Wage' column and store the result in 'wage_amount' column
fifa_df['wage_m'] = fifa_df['Wage'].apply(convert_wage)

# Drop the original 'Wage' column
fifa_df.drop(columns=['Wage'], inplace=True)

# Print the updated DataFrame
print(fifa_df['wage_m'])

0        560.0
1        220.0
2        125.0
3        370.0
4        270.0
         ...  
18974      1.0
18975    500.0
18976    500.0
18977      2.0
18978      1.0
Name: wage_m, Length: 18979, dtype: float64


###Notes for the cell above
* The re.sub(r'[^\d.]|K', '', wage_str) line removes all non-numeric characters and 'K' from the 'Wage' column.
* The resulting string is then converted to a floating-point number, representing the actual wage amount.

###Thirdly, we will transform the 'Release Clause' column

In [ ]:
# Function to convert 'Release Clause' column to numeric format
def convert_release_clause(value_str):
    # Remove non-numeric characters and 'M', then convert to float
    numeric_part = re.sub(r'[^\d.]|M', '', value_str)
    if numeric_part:
        return float(numeric_part)  # Keep the release clause amount in millions (no multiplication)
    else:
        raise ValueError("Invalid 'Release Clause' format: " + value_str)

# Apply the function to 'Release Clause' column and store the result in 'release_clause_amount' column
fifa_df['release_clause_mm'] = fifa_df['Release Clause'].apply(convert_release_clause)

# Drop the original 'Release Clause' column
fifa_df.drop(columns=['Release Clause'], inplace=True)

# Print the updated DataFrame
print(fifa_df['release_clause_mm'])

0        138.4
1         75.9
2        159.4
3        161.0
4        166.5
         ...  
18974     70.0
18975    165.0
18976    131.0
18977     88.0
18978     79.0
Name: release_clause_mm, Length: 18979, dtype: float64


Now, we have converted the value, wage, and release clause columns into floating numbers.

##Question 4: *Can we transform the string values in the 'Hit' column to integers?*

In [ ]:
#Check the datatype of 'Hits' column
fifa_df['Hits'].info

<bound method Series.info of 0        771
1        562
2        150
3        207
4        595
        ... 
18974    NaN
18975    NaN
18976    NaN
18977    NaN
18978    NaN
Name: Hits, Length: 18979, dtype: object>

In [ ]:
print(fifa_df['Hits'])


0        771
1        562
2        150
3        207
4        595
        ... 
18974    NaN
18975    NaN
18976    NaN
18977    NaN
18978    NaN
Name: Hits, Length: 18979, dtype: object


In [ ]:
# Function to convert 'K' notation and floats to integers, replacing NaN with 0
def convert_hits(hit_value):
    if pd.notnull(hit_value):
        if isinstance(hit_value, (float, int)):
            return int(hit_value)
        if isinstance(hit_value, str):
            if 'K' in hit_value:
                return int(float(hit_value.replace('K', '')) * 1000)
            else:
                try:
                    return int(float(hit_value))
                except ValueError:
                    return 0
    return 0

# Convert 'Hits' column to integers while handling 'K' notation, floats, and NaN (replacing NaN with 0)
fifa_df['Hits'] = fifa_df['Hits'].apply(convert_hits)

# Print the updated DataFrame
print(fifa_df['Hits'])

0        771
1        562
2        150
3        207
4        595
        ... 
18974      0
18975      0
18976      0
18977      0
18978      0
Name: Hits, Length: 18979, dtype: int64


###Notes to solution for Question 4

'isinstance()' is a built-in Python function that is used to determine whether an object is an instance of a specific class or data type. It checks if an object belongs to a specified class or data type and returns 'True' if the object is an instance of that class or a sublass, and 'False' otherwise.

Here's the syntax of the 'isinstance()' function:

isinstance(object, classinfo)


In [ ]:
#For Example:
# Check if the variable x is an instance of the int class
x = 5
print(isinstance(x, int))  # Output: True

# Check if the variable y is a float or an integer
y = 3.14
print(isinstance(y, (float, int)))  # Output: True

# Check if a string is an instance of the str class
text = "Hello, World!"
print(isinstance(text, str))  # Output: True


True
True
True
